# Démo des modules Robus

Cett démo permet d'illustrer :

1. la détection de topologie et l'attribution d'id automatique
* l'utilisation d'alias pour accéder directement au module
* la programmation d'interactions entre modules en cachant toute la couche de communication

## Auto-détection de topologie

In [1]:
from pyrobus import Robot

In [2]:
robot = Robot(host='demo-innorobo.local')

Les différents modules connectés sur le robot ainsi que leur topologie sont automatiquement découverts :

In [3]:
robot.modules

[<LED alias="my_led" id=2 state=(0, 0, 0)>,
 <Button alias="my_other_butt" id=3 state=OFF>]

## Accès aux modules grâce à leur alias

On peut accéder aux différents modules directement via leur nom. Les types de modules disponilbes sont :

* Bouton
* LED
* Potentiomètre
* Moteur

### Utilisation d'un module *LED*

On accède au module via son alias :

In [4]:
robot.my_led

<LED alias="my_led" id=2 state=(0, 0, 0)>

On peut directement modifier sa valeur pour changer la couleur de la LED. La synchronization est cachée.

In [5]:
robot.my_led.color = (255, 0, 0)

### Utilisation d'un module *Bouton*

In [6]:
robot.my_other_butt

<Button alias="my_other_butt" id=3 state=OFF>

On peut connaître l'état du bouton (*ON/OFF*) directement depuis l'objet. La valeur est automatiquement mise à jour.

In [7]:
robot.my_other_butt.state

'OFF'

### Utilisation d'un module moteur

In [ ]:
robot.my_motor

On peut changer la position d'un moteur directement :

In [ ]:
robot.my_motor.position = 90

### Utilisation d'un module potentiomètre

In [ ]:
robot.my_pote_tard

On peut connaître la position du potentiomètre via :

In [ ]:
robot.my_pote_tard.position

## Lier deux modules 

Si l'on souhaite changer la valeur de la *LED* lorsque le bouton est appuyé :

In [ ]:
while True:
    if robot.my_butt.state == 'ON':
        robot.my_led.color = (255, 0, 0)
    else:
        robot.my_led.color = (0, 0, 255)

## Gestion d'événements

On peut également faire la même chose en utilisant des événements. Chacun des modules publient des événements qui correspondent à ses changements d'état.

Par exemple, le module bouton peut publier 3 événements :

* *pressed* lorsque le bouton est appuyé
* *released* lorsque le bouton est relaché
* *changed* chaque fois que l'état du bouton change

La liste des événements associés à un module est accessible directement depuis la classe représentant le module.

In [31]:
from pyrobus.modules import Button

Button.possible_events

('changed', 'pressed', 'released')

### Connecter un *Bouton* à une *LED*

Cet exemple d'utilisation d'événement permet de changer la couleur de la LED chaque fois que le bouton est appuyé.

On écrit tout d'abord une fonction qui change la couleur de la *LED* pour une nouvelle couleur aléatoire.

In [ ]:
from random import randint

def change_color(_):
    color = (randint(0, 255), randint(0, 255), randint(0, 255))
    robot.my_led.color = color

On peut ensuite directement l'associer à l'événement *pressed* d'un bouton.

In [15]:
robot.my_other_butt.add_callback(event='pressed', cb=change_color)

### Contrôler un moteur avec un potentiomètre

Dans cet autre exemple, on permet de contrôler la position d'un moteur à l'aide d'un potentiomètre.

In [ ]:
def move_motor(event):
    # The potentiometer gives a value in [0, 100]
    # So we scale it to the motor range: [0, 180]
    robot.my_motor.position = event.new_value * 1.8

In [ ]:
robot.my_pote_tard.add_callback(event='moved', cb=move_motor)